In [109]:
%pylab inline
from itertools import permutations, combinations_with_replacement

Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['e', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [110]:
def choose(n, k):
    num = math.factorial(n)
    den = math.factorial(k)*math.factorial(n - k)
    return num/den

In [111]:
L_max = 0
m = 5
n = 8
#n_k = [round((m + n)*0.75)]

for p in permutations(arange(m, n + m), n):
    V = [p[int(sum(n_k[:i])):int(sum(n_k[:i + 1]))] for i in range(K)]

In [112]:
V

[(12, 11), (10, 9), (8, 7), (6, 5)]

In [113]:
def gen_pi(K = 4, v = 20, acorn = 1234):
    np.random.seed(1234)
    #Taken from Bijan
    
    while True:
        pi_0 = np.random.uniform(size=K) # Generate K Uniform(0,1) random variables
        pisum = np.sum(pi_0) # Calculate the sum of the K generated Uniform(0,1) random variables
        pi = [i/pisum for i in pi_0] # Normalize so that sum(pi) = 1
        Nv = [int(round(i*v)) for i in pi] # Round so that each block has an integer-valued amount of vertices
        if not 0 in Nv and np.sum(Nv)==v: # Make sure no block has 0 vertices and the sum of all vertices is correct
            break
    return pi, Nv # returns the vertex assignment distribution and the number of vertices in each block

In [114]:
def gen_Lambda(K, acorn=1234):
    np.random.seed(acorn)
    Lambda = zeros(shape = (K, K)) # K x K matrix to store adjacency probabilities
    for i in range(K): # for each block
        for j in range(i, K): # for each combination (with replacement)
            Lambda[i, j] = np.random.uniform() # generate a Uniform(0,1) random variable
            Lambda[j, i] = Lambda[i, j] # Lambda is symmetric
                   
    return Lambda # returns a K x K, symmetric matrix where Lambda[i,j] in (0, 1)

In [115]:
def adj_matrix(n, pi, Lambda, acorn = 1234):
    np.random.seed(acorn)
    n = int(n) # Just in case!
    A = np.zeros(shape = (n, n)) # n x n adjcacency matrix
    K = len(pi) # extract the number of blocks in the SBM
    
    i = 0 # start at block indexed with 0
    while i < K: # while the block number is less than the total number of blocks
        for k in range(int(round(n*(sum(pi[:i])))), int(round(n*(sum(pi[:i + 1]))))): # for all vertices in block i
            c = i # start at block i
            while c < K: # while the block number is less than the total number of blocks
                for j in range(int(round(n*(sum(pi[:c])))), int(round(n*(sum(pi[:c + 1]))))): # for all vertices in block c
                    A[k, j] = np.random.binomial(1, Lambda[i, c]) # generates and assigns an edge based on block membership
                    A[j, k] = A[k, j] # A is symmetric
                c += 1
            A[k,k] = 0 # A is hollow
        i += 1
        
    return A # returns an n x n, symmetric and hollow matrix where A[i,j] in {0, 1}

In [ ]:
def gen_seeds(Nv, seed_ratio, acorn = 1234):
    np.random.seed(acorn)
    
    K = len(Nv)
    
    num_seeds = [int(round(seed_ratio*Nv)) for i in range(K)]
    seeds = [[] for i in m]
    for i in range(K):
        for j in range(num_seeds[i]):
            index = np.random.randint(j)
            

In [93]:
n = 12
pi = [0.25, 0.25, 0.25, 0.25]
L = gen_Lambda(4, acorn = 3)
               
A = adj_matrix(n, pi, L, acorn = 3)
L, A

(array([[0.5507979 , 0.70814782, 0.29090474, 0.51082761],
        [0.70814782, 0.89294695, 0.89629309, 0.12558531],
        [0.29090474, 0.89629309, 0.20724288, 0.0514672 ],
        [0.51082761, 0.12558531, 0.0514672 , 0.44080984]]),
 array([[0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.],
        [0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.]]))

In [116]:
def e(A, V):
    K = len(V)
    e = np.zeros(shape = (choose(K, 2) + K, choose(K, 2) + K)) # create an ((K choose 2) + K) x ((K choose 2) + K)
        # to store e[k,l]
        
    e_test = np.zeros(shape = (choose(len(V), 2) + K, choose(len(V), 2) + K))
            
    for i in range(K):
        for j in range(i, K):
            for v in range(V[i]):
                for s in range(V[j]):
                    e[i,j] += A[v, s] 
                    
    for p in combinations_with_replacement(K, 2):
        for v in range(V[i]):
                for s in range(V[j]):
                    e_test[i,j] += A[v, s]
    return e, e_test

In [ ]:




def likelihood_n(V, e, c, p):
    